 <h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9"> Model Building: Loan Default Prediction</span>  </h1>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats       # For Statistical Analysis

import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer            # ML Frameworks
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV

%matplotlib inline 

In [4]:
app_data = pd.read_csv('final_data.csv')

app_data.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_CITY_NOT_LIVE_CITY,...,WEEKDAY_APPR_PROCESS_START_G_THURSDAY,WEEKDAY_APPR_PROCESS_START_G_TUESDAY,WEEKDAY_APPR_PROCESS_START_G_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_G_WEEKEND,ORGANIZATION_TYPE_G_Type2,ORGANIZATION_TYPE_G_Type3,ORGANIZATION_TYPE_G_Type4,ORGANIZATION_TYPE_G_Type5,ORGANIZATION_TYPE_G_Type6,ORGANIZATION_TYPE_G_Type7
0,1,0,202500.0,406597.5,24700.5,0.018801,0,1,10,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,270000.0,1293502.5,35698.5,0.003541,0,2,11,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,67500.0,135000.0,6750.0,0.010032,26,1,9,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,135000.0,312682.5,29686.5,0.008019,0,2,17,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,121500.0,513000.0,21865.5,0.028663,0,1,11,0,...,1,0,0,0,0,0,0,0,1,0


In [5]:
X = app_data.drop(columns = ['TARGET'])    # Independent Variable
Y = app_data['TARGET']                      # Dependent Variable

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Benchmark Model</span>  </h1>

In [6]:
lg_model = LogisticRegression()

lg_model.fit(X, Y)

LogisticRegression()

In [7]:
Y.value_counts(normalize=True)

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

In [8]:
Y_pred = lg_model.predict(X)

In [9]:
lg_model.coef_   ## Gives the coefficients used for repective columns in independent variable

array([[-1.18736018e-04, -3.50549301e-06, -2.25696736e-07,
        -1.72805237e-06, -1.13648004e-05, -1.08627387e-03,
        -1.05088529e-03, -6.66768818e-03,  2.83739701e-05,
        -2.40088411e-06, -1.41221465e-06, -1.37911103e-05,
        -1.04217185e-04, -1.17201487e-04, -6.65031695e-04,
        -3.09554570e-02, -5.93407817e-03, -1.89529978e-02,
        -1.85511145e-04,  8.85690345e-05, -8.86998358e-05,
        -4.03199133e-04, -2.35575743e-04, -8.11309240e-05,
        -5.51857609e-04, -2.23160231e-04, -2.81569528e-06,
        -5.07878452e-04, -2.40539805e-05, -5.09445849e-04,
        -2.17999516e-05, -5.61038796e-04, -5.79941057e-04,
        -5.72791850e-04, -5.69870673e-04, -5.48129619e-04,
        -4.53321029e-06, -4.51406191e-04,  2.89582685e-07,
        -2.35385840e-04, -1.36152315e-04, -9.48144494e-06,
        -4.07281617e-04, -4.96848241e-05, -1.25042975e-04,
        -5.01549510e-06,  3.73694526e-06, -2.58242191e-04,
         1.38541093e-06, -2.58568527e-04, -1.02247629e-0

In [10]:
lg_model.intercept_

array([-0.00055345])

In [11]:
accuracy_score(Y, Y_pred)   ## Not good because, right now our classes are imbalanced, 
                            ## i.e., more number of 0's and less number of 1s
    
    ## Here, the threshold is already defined

0.9192711805431351

In [12]:
confusion_matrix(Y, Y_pred, labels = [1, 0])   ## Very high baisness in the model, good at predicting 0s not 1s

array([[     0,  24825],
       [     0, 282686]])

Here, accuracy is not a good criteria.

In [13]:
f1_score(Y, Y_pred)

0.0

In [14]:
Y.value_counts(normalize = True)

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

Here, we are having a proportion mismatch, as maximum number of values are 0s not a balance of 0 and 1.

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Changing Proporation of Target Class</span>  </h1>

In [43]:
data_1 = app_data[app_data['TARGET'] == 1]
data_2 = app_data[app_data['TARGET'] == 0].sample(frac = 0.07)

In [44]:
app_data[app_data['TARGET'] == 0].describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET,282686.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
CNT_CHILDREN,282686.0,0.412153,0.711947,0.0,0.0,0.0,1.0,4.0
AMT_INCOME_TOTAL,282686.0,166599.255677,83606.103265,25650.0,112500.0,148500.0,202500.0,472500.0
AMT_CREDIT,282686.0,602648.282002,406845.907710,45000.0,270000.0,517788.0,810000.0,4050000.0
AMT_ANNUITY,282686.0,27163.527386,14658.003451,1615.5,16456.5,24876.0,34749.0,258025.5
...,...,...,...,...,...,...,...,...
ORGANIZATION_TYPE_G_Type3,282686.0,0.229141,0.420281,0.0,0.0,0.0,0.0,1.0
ORGANIZATION_TYPE_G_Type4,282686.0,0.427913,0.494777,0.0,0.0,0.0,1.0,1.0
ORGANIZATION_TYPE_G_Type5,282686.0,0.248382,0.432075,0.0,0.0,0.0,0.0,1.0
ORGANIZATION_TYPE_G_Type6,282686.0,0.088055,0.283376,0.0,0.0,0.0,0.0,1.0


In [45]:
data_2.describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET,19788.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0
CNT_CHILDREN,19788.0,0.414645,0.710861,0.0,0.0,0.0,1.000,4.0
AMT_INCOME_TOTAL,19788.0,166785.937038,84182.916001,27000.0,112500.0,148500.0,202500.000,472500.0
AMT_CREDIT,19788.0,603703.784263,408215.050294,45000.0,270000.0,512446.5,814041.000,4050000.0
AMT_ANNUITY,19788.0,27222.752729,14829.992385,1993.5,16456.5,24714.0,34948.125,230161.5
...,...,...,...,...,...,...,...,...
ORGANIZATION_TYPE_G_Type3,19788.0,0.224985,0.417583,0.0,0.0,0.0,0.000,1.0
ORGANIZATION_TYPE_G_Type4,19788.0,0.430311,0.495132,0.0,0.0,0.0,1.000,1.0
ORGANIZATION_TYPE_G_Type5,19788.0,0.247322,0.431466,0.0,0.0,0.0,0.000,1.0
ORGANIZATION_TYPE_G_Type6,19788.0,0.090206,0.286484,0.0,0.0,0.0,0.000,1.0


There is not much difference in terms of Statistical distribution between the Original 0s data and the different proportion of the same data.

In [46]:
data_1.shape, data_2.shape

((24825, 62), (19788, 62))

In [47]:
data_full = pd.concat([data_1, data_2], axis = 0)
data_full.shape

(44613, 62)

In [48]:
data_full = pd.concat([data_1, data_2], axis = 0)
X = data_full.drop(columns = ['TARGET'])
Y = data_full['TARGET']

In [49]:
Y.value_counts(normalize = True)         ## Change in the proportion of the 0s and 1s in the final data

1    0.556452
0    0.443548
Name: TARGET, dtype: float64

In [50]:
lg_model.fit(X, Y)

LogisticRegression(class_weight='balanced')

In [51]:
Y_pred = lg_model.predict(X)

In [52]:
confusion_matrix(Y, Y_pred, labels = [1, 0])

array([[10488, 14337],
       [ 7124, 12664]])

In [53]:
print(f'The accuracy and f1 score on the training data is {accuracy_score(Y, Y_pred)} and {f1_score(Y, Y_pred)} respectively.')

The accuracy and f1 score on the training data is 0.5189518750140094 and 0.49428564695902166 respectively.


The accuracy score and the f1 score have improved comparing to the previous base (dumb) model.

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Using Balanced Class</span>  </h1>

In [54]:
lg_model = LogisticRegression(class_weight = 'balanced')   ## Automatically takes care of class imbalance

In [55]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.3, shuffle = True, stratify = Y)

In [56]:
ytrain.value_counts(normalize = True)
ytest.value_counts(normalize = True)

1    0.556485
0    0.443515
Name: TARGET, dtype: float64

In [57]:
lg_model = LogisticRegression(class_weight = 'balanced')
lg_model.fit(xtrain, ytrain)

LogisticRegression(class_weight='balanced')

In [58]:
Y_train_pred = lg_model.predict(xtrain)
Y_test_pred = lg_model.predict(xtest)

In [59]:
confusion_matrix(ytrain, Y_train_pred, labels = [1,0]) 

array([[7394, 9983],
       [4322, 9530]])

In [60]:
confusion_matrix(ytest, Y_test_pred, labels = [1,0]) 

array([[3102, 4346],
       [1878, 4058]])

In [61]:
print(f'The accuracy and f1 score on the training data is {accuracy_score(ytrain,Y_train_pred)} and {f1_score(ytrain,Y_train_pred)} respectively.')
print('')
print(f'The accuracy and f1 score on the testing data is {accuracy_score(ytest,Y_test_pred)} and {f1_score(ytest,Y_test_pred)} respectively.')

The accuracy and f1 score on the training data is 0.5419321784239008 and 0.5083009658680783 respectively.

The accuracy and f1 score on the testing data is 0.5349671249252839 and 0.499195365304152 respectively.


Accuracy and performance of the this model with balanced class_weight has improved from the previous model with varied proporation of the target class.

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Hyperparameter Tuning</span>  </h1>

In [62]:
clf = LogisticRegression(solver = 'saga')

In [63]:
search_space = {'penalty':['l1', 'l2', 'elasticnet', 'none'],
                    'tol':[.00001,.0001,.001,.01,.7],
                    'C':[1,2,3,4], 'fit_intercept':[True,False], 'solver':['saga'],'l1_ratio':[.001,1]}
search_space

{'penalty': ['l1', 'l2', 'elasticnet', 'none'],
 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.7],
 'C': [1, 2, 3, 4],
 'fit_intercept': [True, False],
 'solver': ['saga'],
 'l1_ratio': [0.001, 1]}

In [64]:
random_clf = RandomizedSearchCV(estimator=clf, param_distributions=search_space, n_iter=5, scoring='f1', cv=2)

In [65]:
random_clf.fit(xtrain, ytrain)

RandomizedSearchCV(cv=2, estimator=LogisticRegression(solver='saga'), n_iter=5,
                   param_distributions={'C': [1, 2, 3, 4],
                                        'fit_intercept': [True, False],
                                        'l1_ratio': [0.001, 1],
                                        'penalty': ['l1', 'l2', 'elasticnet',
                                                    'none'],
                                        'solver': ['saga'],
                                        'tol': [1e-05, 0.0001, 0.001, 0.01,
                                                0.7]},
                   scoring='f1')

In [66]:
random_clf.best_params_

{'tol': 0.01,
 'solver': 'saga',
 'penalty': 'l2',
 'l1_ratio': 0.001,
 'fit_intercept': True,
 'C': 1}

In [67]:
pd.DataFrame(random_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tol,param_solver,param_penalty,param_l1_ratio,param_fit_intercept,param_C,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.626641,0.020145,0.006076,0.000832,0.00001,saga,none,0.001,True,4,"{'tol': 1e-05, 'solver': 'saga', 'penalty': 'n...",0.708660,0.696104,0.702382,0.006278,3
1,0.768329,0.000988,0.006407,0.000004,0.00001,saga,elasticnet,1,True,3,"{'tol': 1e-05, 'solver': 'saga', 'penalty': 'e...",0.708660,0.696104,0.702382,0.006278,3
2,0.788774,0.023701,0.006212,0.000056,0.0001,saga,l1,1,False,1,"{'tol': 0.0001, 'solver': 'saga', 'penalty': '...",0.708660,0.696104,0.702382,0.006278,3
3,0.263170,0.005996,0.006213,0.000052,0.01,saga,l2,0.001,True,1,"{'tol': 0.01, 'solver': 'saga', 'penalty': 'l2...",0.712188,0.700661,0.706425,0.005763,1
4,0.766814,0.005769,0.005617,0.000679,0.00001,saga,l1,0.001,True,2,"{'tol': 1e-05, 'solver': 'saga', 'penalty': 'l...",0.708660,0.696162,0.702411,0.006249,2


In [68]:
ytrain_pred = random_clf.predict(xtrain)
ytest_pred = random_clf.predict(xtest)

In [69]:
confusion_matrix(ytest, Y_test_pred, labels = [1,0]) 

array([[3102, 4346],
       [1878, 4058]])

In [70]:
print(f'The accuracy and f1 score on the training data is {accuracy_score(ytrain,ytrain_pred)} and {f1_score(ytrain,ytrain_pred)} respectively.')
print('')
print(f'The accuracy and f1 score on the testing data is {accuracy_score(ytest,ytest_pred)} and {f1_score(ytest,ytest_pred)} respectively.')


The accuracy and f1 score on the training data is 0.5610810464632233 and 0.7043165002049314 respectively.

The accuracy and f1 score on the testing data is 0.5617901972504483 and 0.7055870689222429 respectively.


Hence, using Hyperparameter tuning our model's accuracy as well as f1 score has improved.

In conclusion, we observed that:

$$\textbf{Model_{Hyperparamater} > Model_{Balanced} > Model_{Baseline}}$$

In terms of performance and F1 Score.